# 🎥 Floutage automatique de vidéo par vision par ordinateur

## Objectif du projet
Ce projet vise à protéger la vie privée dans les vidéos numériques en
détectant et en floutant automatiquement des éléments sensibles tels que :
- les visages,
- les plaques d’immatriculation,
- les écrans (ordinateur, téléphone, télévision).

Le traitement repose sur des techniques de vision par ordinateur et
d’apprentissage profond appliquées image par image sur une séquence vidéo.


### Importation des bibliothèques

Cette cellule permet d’importer les bibliothèques nécessaires au traitement vidéo et à la détection d’objets :

- **OpenCV (cv2)** : utilisée pour la lecture, le traitement et la manipulation des images et des vidéos.
- **NumPy** : permet de gérer efficacement les tableaux et les calculs numériques.
- **YOLO (Ultralytics)** : modèle de détection d’objets basé sur le deep learning, utilisé pour identifier automatiquement les objets sensibles.
- **Matplotlib** : utilisée pour l’affichage et la visualisation des images.
- **IPython.display** : permet d’afficher directement des vidéos dans le notebook Jupyter.


In [16]:
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
from IPython.display import Video, display
import os


### Chargement des modèles de détection

Cette cellule initialise les modèles de détection d’objets basés sur YOLO, chacun étant spécialisé dans un type d’éléments sensibles :

- **Modèle de détection des visages** : utilisé pour identifier automatiquement les visages présents dans la vidéo.
- **Modèle ALPR (Automatic License Plate Recognition)** : permet de détecter les plaques d’immatriculation afin de préserver la confidentialité.
- **Modèle COCO** : modèle généraliste entraîné sur le jeu de données COCO, utilisé ici pour détecter certains objets sensibles.

La liste `COCO_TARGETS` définit les catégories d’objets à flouter, telles que les ordinateurs portables, les téléphones portables et les écrans de télévision.


In [2]:
face_model = YOLO("yolov8s-face-lindevs.pt")
alpr_model = YOLO("best.pt")
coco_model = YOLO("yolov8n.pt")

COCO_TARGETS = ["laptop", "cell phone", "tv"]


### Fonctions utilitaires pour le floutage

Cette cellule définit deux fonctions essentielles utilisées lors du traitement vidéo :

- **`clamp_bbox`** :  
  Cette fonction ajuste les coordonnées d’une boîte englobante afin qu’elles restent à l’intérieur des dimensions de l’image. Elle permet d’éviter les erreurs liées aux débordements et ignore les régions invalides.

- **`flouter_roi`** :  
  Cette fonction applique un floutage gaussien sur une région d’intérêt (ROI) correspondant à un objet détecté.  
  L’intensité du floutage est automatiquement adaptée à la taille de la région, et deux modes sont possibles :
  - floutage rectangulaire classique,
  - floutage elliptique, particulièrement adapté au floutage des visages.


In [3]:
def clamp_bbox(bbox, shape):
    x1, y1, x2, y2 = map(int, bbox)
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(shape[1]-1, x2)
    y2 = min(shape[0]-1, y2)
    if x2 <= x1 or y2 <= y1:
        return None
    return x1, y1, x2, y2


def flouter_roi(frame, bbox, intensite=3, ellipse=False):
    x1, y1, x2, y2 = bbox
    roi = frame[y1:y2, x1:x2]
    if roi.size == 0:
        return

    k = max(15, ((x2-x1)//3) | 1) * intensite
    flou = cv2.GaussianBlur(roi, (k, k), 0)

    if ellipse:
        mask = np.zeros(roi.shape[:2], dtype=np.uint8)
        cv2.ellipse(
            mask,
            ((x2-x1)//2, (y2-y1)//2),
            ((x2-x1)//2, (y2-y1)//2),
            0, 0, 360, 255, -1
        )
        roi[mask == 255] = flou[mask == 255]
    else:
        roi[:] = flou


### Chargement et configuration de la vidéo

Cette cellule permet d’ouvrir la vidéo d’entrée et d’extraire ses principales propriétés, telles que la largeur, la hauteur et le nombre d’images par seconde (FPS).

Un objet `VideoWriter` est ensuite initialisé afin d’enregistrer la vidéo de sortie après application du floutage des éléments sensibles, tout en conservant les caractéristiques originales de la vidéo.


In [10]:
video_path = "C:\\Users\\fatim\\OneDrive\\Bureau\\Voiture.mp4"  # chemin vers la vidéo
cap = cv2.VideoCapture(video_path)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter(
    "output_floute.mp4",
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (width, height)
)


### Traitement des images vidéo et application du floutage

Cette cellule réalise le traitement principal de la vidéo image par image.  
Chaque frame est redimensionnée afin d’accélérer la détection des objets par les modèles YOLO.

Les modèles de détection (visages, plaques d’immatriculation et objets COCO ciblés) sont ensuite appliqués successivement.  
Pour chaque objet détecté, les coordonnées sont recalculées à l’échelle originale de la vidéo, puis vérifiées avant d’appliquer un floutage adapté à la région concernée.

Les frames modifiées sont enfin enregistrées dans une nouvelle vidéo de sortie, garantissant la protection des éléments sensibles tout en conservant la qualité globale de la scène.


In [12]:
frame_id = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1

    small = cv2.resize(frame, (640, int(640 * height / width)))

    for model, ellipse, labels in [
        (face_model, True, None),
        (alpr_model, False, None),
        (coco_model, False, COCO_TARGETS)
    ]:
        results = model(small, conf=0.4, verbose=False)[0]

        for box, cls in zip(results.boxes.xyxy, results.boxes.cls):
            if labels and model.names[int(cls)] not in labels:
                continue

            bbox = [
                box[0] * width / 640,
                box[1] * height / small.shape[0],
                box[2] * width / 640,
                box[3] * height / small.shape[0]
            ]

            safe = clamp_bbox(bbox, frame.shape)
            if safe:
                flouter_roi(frame, safe, intensite=3, ellipse=ellipse)

    out.write(frame)

cap.release()
out.release()


### Traitement des images vidéo et application du floutage

Cette cellule réalise le traitement principal de la vidéo image par image.  
Chaque frame est redimensionnée afin d’accélérer la détection des objets par les modèles YOLO.

Les modèles de détection (visages, plaques d’immatriculation et objets COCO ciblés) sont ensuite appliqués successivement.  
Pour chaque objet détecté, les coordonnées sont recalculées à l’échelle originale de la vidéo, puis vérifiées avant d’appliquer un floutage adapté à la région concernée.

Les frames modifiées sont enfin enregistrées dans une nouvelle vidéo de sortie, garantissant la protection des éléments sensibles tout en conservant la qualité globale de la scène.


In [21]:
from IPython.display import display, Markdown

video_path = r"C:\Users\fatim\OneDrive\Bureau\Projects_S3\TraitementVideo\output_floute.mp4"

# Message simple
display(Markdown(f"✅ La vidéo a été enregistrée avec succès : `{video_path}`"))


✅ La vidéo a été enregistrée avec succès : `C:\Users\fatim\OneDrive\Bureau\Projects_S3\TraitementVideo\output_floute.mp4`

## Discussion et pistes d’amélioration

- Accélération du traitement à l’aide du GPU
- Utilisation d’un tracker multi-objets plus robuste
- Amélioration de la détection des plaques lointaines
- Passage à un traitement en temps réel


## Remarque complémentaire

En complément de ce notebook Jupyter, une application interactive a été
développée à l’aide de Streamlit.  
Cette application repose sur le même pipeline de traitement vidéo et permet
une utilisation plus intuitive (import de vidéo, choix des options de floutage,
visualisation avant/après).

Lien vers l’application Streamlit :  
👉 https://lien-de-ton-app.streamlit.app


### Conclusion technique

Ce notebook a permis de mettre en œuvre une application complète de traitement vidéo basée sur la vision par ordinateur et le deep learning. En s’appuyant sur plusieurs modèles YOLO spécialisés, le système est capable de détecter automatiquement différents éléments sensibles tels que les visages, les plaques d’immatriculation et certains objets du quotidien.

Le traitement image par image, combiné à un redimensionnement optimisé et à une gestion sécurisée des régions d’intérêt, garantit un bon compromis entre précision de détection et performance d’exécution. L’application du floutage gaussien, ajusté dynamiquement à la taille des objets détectés, assure une protection efficace de la vie privée tout en préservant la lisibilité globale de la vidéo.

Cette approche modulaire et extensible constitue une base solide pour des applications réelles telles que l’anonymisation de vidéos, l’observation intelligente ou les systèmes de sécurité respectueux de la confidentialité.
